In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [2]:
file_path = "datasets\datos_CB.csv"
student_data = pd.read_csv(file_path)
student_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97341 entries, 0 to 97340
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  97341 non-null  int64  
 1   Unnamed: 0    97341 non-null  int64  
 2   ciclo         97341 non-null  int64  
 3   Cod.Asign     97341 non-null  int64  
 4   Asignatura    97341 non-null  object 
 5   Cod.Car.Sec   97341 non-null  object 
 6   Cod.Curso     97341 non-null  int64  
 7   Convocatoria  97341 non-null  int64  
 8   Anho          97341 non-null  int64  
 9   Semestre      97341 non-null  int64  
 10  Aprobado      97341 non-null  int64  
 11  Anho.Firma    97341 non-null  int64  
 12  Primer.Par    97341 non-null  int64  
 13  Segundo.Par   97341 non-null  int64  
 14  AOT           97341 non-null  int64  
 15  Primer.Rec    97341 non-null  int64  
 16  Segundo.Rec   97341 non-null  int64  
 17  Nota.Final    97341 non-null  object 
 18  Asis          97341 non-nu

In [3]:
columns = ['Unnamed: 0.1', 'Unnamed: 0', "Cod.Asign", 'Asis', '4P_60',"Cod.Curso", "Cod.Car.Sec", "Semestre", "id_anony"] #el id_anony podriamos usar despues
if columns[0] in student_data:
    student_data.drop(columns,axis=1, inplace= True)
student_data.head()

,ciclo,Asignatura,Convocatoria,Anho,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final
0,1,CALCULO 3,1,2012,0,0,0,0,0,0,0,[0]
1,1,FISICA 4,1,2012,0,2012,19,13,9,0,0,[0]
2,1,FISICA 4,1,2012,1,2012,17,24,10,0,0,[3]
3,1,FISICA 4,1,2012,0,0,0,0,0,0,0,[0]
4,1,FISICA 4,1,2012,0,0,16,12,8,0,0,[0]


In [4]:
#todo esto puede ser un diccionario con numpy unique y queda re loco
course = "ESTATICA"
condition = student_data["Asignatura"] == course
course_data = student_data[condition]
print(course_data.head())
course_data.drop(["Asignatura"],axis=1, inplace = True)
course_data.head()

      ciclo Asignatura  Convocatoria  Anho  Aprobado  Anho.Firma  Primer.Par  \
5258      2   ESTATICA             1  2012         0           0           2   
5259      2   ESTATICA             1  2012         0           0           6   
5260      2   ESTATICA             1  2012         1        2012          12   
5261      2   ESTATICA             1  2012         0           0           1   
5262      2   ESTATICA             1  2012         0           0           0   

      Segundo.Par  AOT  Primer.Rec  Segundo.Rec Nota.Final  
5258           13    0           0            0        [0]  
5259           11    0           0            0        [0]  
5260           24    0           0            0        [2]  
5261            1    0           0            0        [0]  
5262            0    0           0            0        [0]  


C:\Users\Guana\AppData\Local\Temp\ipykernel_22288\1374848725.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_data.drop(["Asignatura"],axis=1, inplace = True)


,ciclo,Convocatoria,Anho,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final
5258,2,1,2012,0,0,2,13,0,0,0,[0]
5259,2,1,2012,0,0,6,11,0,0,0,[0]
5260,2,1,2012,1,2012,12,24,0,0,0,[2]
5261,2,1,2012,0,0,1,1,0,0,0,[0]
5262,2,1,2012,0,0,0,0,0,0,0,[0]


In [5]:
%%capture
print(np.unique(course_data["Anho"]))

In [6]:
course_data["Primer.Par"].describe()

count    4824.000000
mean        9.337894
std         5.849116
min         0.000000
25%         5.000000
50%        10.000000
75%        14.000000
max        24.000000
Name: Primer.Par, dtype: float64

In [14]:
approved = len(course_data[course_data["Aprobado"]==1])
not_approved = len(course_data[course_data["Aprobado"] == 0])
total = len(course_data["Aprobado"])
print(f"Approved: {approved}\nNot approved: {not_approved}\nTotal: {total}")

Approved: 1818
Not approved: 3006
Total: 4824


In [8]:
df_train, df_test= train_test_split(course_data, test_size=0.2, random_state=0)
print(df_train.shape)
print(df_test.shape)


# Saving our training and testing data sets
'''
# Training Set
train_save_path = 'student_train.csv'
train_df.to_csv(train_save_path, index=False)

# Testing Set
test_save_path = os.path.join(os.getcwd(), 'student', 'student-por-test.csv')
test_df.to_csv(test_save_path, index=False)
'''

(3859, 11)
(965, 11)


"\n# Training Set\ntrain_save_path = 'student_train.csv'\ntrain_df.to_csv(train_save_path, index=False)\n\n# Testing Set\ntest_save_path = os.path.join(os.getcwd(), 'student', 'student-por-test.csv')\ntest_df.to_csv(test_save_path, index=False)\n"

In [9]:
# Dummy Encoding the Data
# Coverting categorical data to numerical dummy variables
#train_dummies_df = pd.get_dummies(df_train)
#test_dummies_df = pd.get_dummies(df_test)

# Separating training data into predictors and predictand
use_cols = [column for column in df_train.columns if column not in ['Aprobado']]
train_x = df_train[use_cols]
train_y = df_train['Aprobado']

# Separating testing data into predictors and predictand
test_x = df_test[use_cols]
test_y = df_test['Aprobado']

In [10]:
train_x.head()
new_train_x = train_x.drop(['Nota.Final'],axis=1)

In [11]:
train_y.head()

5757     0
58811    0
59840    0
46862    0
36301    1
Name: Aprobado, dtype: int64

In [12]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3859 entries, 5757 to 58585
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ciclo         3859 non-null   int64 
 1   Convocatoria  3859 non-null   int64 
 2   Anho          3859 non-null   int64 
 3   Anho.Firma    3859 non-null   int64 
 4   Primer.Par    3859 non-null   int64 
 5   Segundo.Par   3859 non-null   int64 
 6   AOT           3859 non-null   int64 
 7   Primer.Rec    3859 non-null   int64 
 8   Segundo.Rec   3859 non-null   int64 
 9   Nota.Final    3859 non-null   object
dtypes: int64(9), object(1)
memory usage: 331.6+ KB


In [39]:
default_LR = LogisticRegression(max_iter=1000)

# Fitting the model
model = default_LR.fit(new_train_x, train_y)

# Plotting the CV Accuracy with respect to C values
#fig, axs = plt.subplots(1, 1, figsize=(14, 7))
#axs.plot(new_train_x, train_y)
#axs.grid()

predicted_y = model.predict(test_x.drop(['Nota.Final'],axis=1))

score = model.score(test_x.drop(['Nota.Final'],axis=1),test_y)
print(f"Model score for {course.lower()}: {score}")
print(f"Parameters: \n{model.feature_names_in_}")
print(f"iterations: {model.n_iter_}")
print(f"coefs: {model.coef_}")
#logRegSDAccuracyNo0s = accuracy_score(logRegCVSDNo0s.predict(test_x_standardized), test_y)

Model score for estatica: 0.8797927461139896
Parameters: 
['ciclo' 'Convocatoria' 'Anho' 'Anho.Firma' 'Primer.Par' 'Segundo.Par'
 'AOT' 'Primer.Rec' 'Segundo.Rec']
iterations: [211]
coefs: [[ 0.13401082 -0.67726307 -0.00428443  0.00359478  0.09151401  0.08353366
   0.08441298  0.00871577 -0.02400777]]


In [50]:
SGDC = SGDClassifier(loss='log_loss')

model = SGDC.fit(new_train_x, train_y)

predicted_y = model.predict(test_x.drop(['Nota.Final'],axis=1))

score = model.score(test_x.drop(['Nota.Final'],axis=1),test_y)
print(f"Model score for {course.lower()}: {score}")
print(f"Parameters: \n{model.feature_names_in_}")
print(f"iterations: {model.n_iter_}")
print(f"coefs: {model.coef_}")

Model score for estatica: 0.8559585492227979
Parameters: 
['ciclo' 'Convocatoria' 'Anho' 'Anho.Firma' 'Primer.Par' 'Segundo.Par'
 'AOT' 'Primer.Rec' 'Segundo.Rec']
iterations: 87
coefs: [[   51.46521734   -69.86272226  -100.49131537   120.60443422
   2803.49018441  5492.91344778  1350.76261995   -38.17577287
  -1820.89911265]]


In [70]:
SGDC = SGDClassifier(loss="perceptron", eta0=1, learning_rate="constant", penalty='l2')

model = SGDC.fit(new_train_x, train_y)

predicted_y = model.predict(test_x.drop(['Nota.Final'],axis=1))

score = model.score(test_x.drop(['Nota.Final'],axis=1),test_y)
print(f"Model score for {course.lower()}: {score}")
print(f"Parameters: \n{model.feature_names_in_}")
print(f"iterations: {model.n_iter_}")
print(f"coefs: {model.coef_}")

Model score for estatica: 0.8559585492227979
Parameters: 
['ciclo' 'Convocatoria' 'Anho' 'Anho.Firma' 'Primer.Par' 'Segundo.Par'
 'AOT' 'Primer.Rec' 'Segundo.Rec']
iterations: 12
coefs: [[   74.5663602    -79.52057807  -253.86763428  1086.97008958
   3693.36765102  7708.69867153  1703.05239136 -2060.45067953
  -2829.74758132]]
